# Create main Timeseries object
The following cell combines all the parsed csv's (one per month) from the given folder into a single Timeseries object.

In [6]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
import numpy as np
import pandas as pd
import glob
register_matplotlib_converters()


# ------ Path containing CSVs to be parsed and merged ------ #
PATH = "runescape"
# ---------------------------------------------------------- #

# ------ Path to the manually labeled file ----------------- #
REAL_LABELS_FILE = "minecraft_new_hypixel/labeled/hypixel_all_labeled.csv"
# ---------------------------------------------------------- #


# ------ Resampling frequency ------------------------------ #
RESAMPLE_FREQ = 2 # MINUTES. New sample every X minutes.
# ---------------------------------------------------------- #


all_files = glob.glob(PATH + "/*.csv")

li = []
for filename in all_files:
    ts = pd.read_csv(filename, header=0, parse_dates=[0], index_col=0, squeeze=True)
    li.append(ts)

final_ts = pd.concat(li, axis=0, ignore_index=False)
final_ts = final_ts.sort_index()

# Resample using globally defined freq
final_ts = final_ts.resample(str(RESAMPLE_FREQ)+"min", level=0).mean()

#Non-interpolated dataset:
raw_ts = final_ts

# Interpolated final Timeseries:
final_ts = final_ts.interpolate()


# ------------------ OPTIONAL: Truncate dataset to focus on specific period ------------------ #

#final_ts = final_ts.truncate(pd.Timestamp(2007, 10, 1), pd.Timestamp(2015, 3, 22)) # All data
#final_ts = final_ts.truncate(pd.Timestamp(2014, 7, 1), pd.Timestamp(2014, 7, 30))  # Test month
final_ts = final_ts.truncate(pd.Timestamp(2013, 5, 1), pd.Timestamp(2013, 7, 30))  # Test few months

# -------------------------------------------------------------------------------------------- #

# Heatmap of player counts

In [7]:
%matplotlib widget

import datetime
import calendar
import matplotlib.dates as mdates

df = pd.DataFrame(final_ts)
df.insert(1, 'weekday', df.index.weekday)
df.insert(2, 'time', df.index.to_series().apply(lambda x: x.replace(day=1, month=1, year=2020)))

lst = []

hours = range(0,24)
weekdays = list(reversed(calendar.day_name))

for wd in range(6, -1, -1):
    ls = []
    for tm in range(0,24):
        ls.append(df['0'].where(np.logical_and(df['time'].dt.hour == tm, df['weekday'] == wd)).sum())
    lst.append(ls)

arr = np.array(lst)

fig, ax = plt.subplots()
im = ax.imshow(arr, interpolation="nearest")

ax.set_xticks(np.arange(len(hours)))
ax.set_yticks(np.arange(len(weekdays)))

ax.set_xticklabels(hours)
ax.set_yticklabels(weekdays)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

ax.set_title("Heatmap of total player count by hour")
fig.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Remove Seasonality and Trend from the time series
This cell will decompose the time series into the trend and seasonal components. These are then removed from the original dataset to be left with only the residuals.
Two separate methods are used. The very fast but less thorough 'seasonal_decompose', and the slow but good STL method.
The first method does two passes, to remove both daily and weekly seasonality. This is impractical for the STL method due to time.

In [8]:
%matplotlib widget
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.nonparametric.smoothers_lowess import lowess
from statsmodels.tsa.seasonal import STL

SAMPLES_PER_DAY = int((24*60) / RESAMPLE_FREQ) # Amount of samples per day is: minutes per day / sample period in minutes.


# ------------ Time Series Decomposition using 'seasonal_decompose' ------------
result_decomp = seasonal_decompose(final_ts, period=SAMPLES_PER_DAY, model="additive")
detrended1 = final_ts.values - result_decomp.trend # Detrend
deseasonalized1 = detrended1.values - result_decomp.seasonal # Deseasonalize
# Second pass for weekly seasonality
result_decomp2 = seasonal_decompose(deseasonalized1.fillna(0), period=SAMPLES_PER_DAY*7, model="additive")
detrended2 = deseasonalized1.values - result_decomp2.trend
deseasonalized_decompose = detrended2.values - result_decomp2.seasonal
# ------------------------------------------------------------------------------


# ------------ Time Series Decomposition using 'STL' ---------------------------
result_stl = STL(final_ts, period=SAMPLES_PER_DAY).fit()
detrended = final_ts.values - result_stl.trend # Detrend
deseasonalized_stl = detrended.values - result_stl.seasonal # Deseasonalize
# ------------------------------------------------------------------------------


# --- CHOOSE which result to use from here on out --- #
deseasonalized = deseasonalized_stl
# --------------------------------------------------- #


# --- Plot the results ---
plt.subplot(321)
plt.plot(result_stl.trend)
ax = plt.gca()
ax.legend(['Trend'])
plt.subplot(323)
plt.plot(result_stl.seasonal)
ax = plt.gca()
ax.legend(['Seasonality'])
plt.subplot(325)
plt.plot(result_stl.resid)
ax = plt.gca()
ax.legend(['Residuals'])

plt.subplot(122)
plt.plot(deseasonalized_decompose, linewidth=1, color='#E14932')
plt.plot(deseasonalized_stl, linewidth=1, color="orange")
plt.plot(final_ts, color="blue")
ax = plt.gca()
ax.legend(['seasonal_decompose', 'STL', 'Raw Data'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
%matplotlib widget
# --- Plot the results ---
plt.subplot(321)
plt.plot(result_stl.trend)
ax = plt.gca()
ax.legend(['Trend'])
plt.subplot(323)
plt.plot(result_stl.seasonal, linewidth=0.1)
ax = plt.gca()
ax.legend(['Seasonality'])
plt.subplot(325)
plt.plot(result_stl.resid, linewidth=0.5)
ax = plt.gca()
ax.legend(['Residuals'])

plt.subplot(122)
plt.plot(deseasonalized_decompose, linewidth=1, color='#E14932')
plt.plot(deseasonalized_stl, linewidth=1, color="orange")
plt.plot(final_ts, color="blue")
ax = plt.gca()
ax.legend(['seasonal_decompose', 'STL', 'Raw Data'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Detect Failures (using Z-scores + peak expansion + bonus peaks)
Find first point with high z-score.

Backtrack point by point, until the diff of this point is suddenly much lower. This is start of peak. Save the Y value of this point.

Find last point with high-zscore.

Look ahead point by point until Y value comes close to the saved Y value (start of peak). This is done via a percentage of the normalized peak height. (Top of peak minus min value of peak) If the course starts reversing (going down again before reaching previous peak height), also stop the lookahead.

To increase recall, points where there is a sudden sharp drop in diff are also marked as peaks, regardless of whether there is a peak in the deseasonalized data or not. 

In [10]:
%matplotlib widget
from sklearn.metrics import confusion_matrix
from scipy import stats

# -------- Algorithm Parameters ----------------------- #
WINDOW_SIZE = 4333#675            # Number of samples in the window.
Z_THRESHOLD = 4.62#5.5            # Minimum Z-Score for point to be marked as peak.

MAXIMUM_LOOKAHEAD = 720      # Stop expanding peak after X samples if start/end not found yet.
PEAK_START_DIFF_FACTOR = 2.28#4   # Peak start defined as when diff is suddenly at least N times smaller.
PEAK_END_RECOVER_PERC = 0.83#0.65 # Peak end defined when N% of the pre-peak value has been regained.

BONUS_DIFF_THRES = final_ts.mean() / 15.3#19 # The minimum Diff value for a point to be marked as a bonus peak


WINDOW_SIZE = 4784#675            # Number of samples in the window.
Z_THRESHOLD = 4.86#5.5            # Minimum Z-Score for point to be marked as peak.

MAXIMUM_LOOKAHEAD = 720      # Stop expanding peak after X samples if start/end not found yet.
PEAK_START_DIFF_FACTOR = 2.06#4   # Peak start defined as when diff is suddenly at least N times smaller.
PEAK_END_RECOVER_PERC = 0.84#0.65 # Peak end defined when N% of the pre-peak value has been regained.

BONUS_DIFF_THRES = final_ts.mean() / 15.3#19 # The minimum Diff value for a point to be marked as a bonus peak


#[4784, 4.863266543366097, 5757, 2.0560282392446982, 0.8423319799948153]
# -------- PeakData dataframe shape definition -------- #
peakData = pd.DataFrame(columns=['group', 'timespan', 'magnitude', 'drop', 'shape', 'startdate', 'enddate'])
# ----------------------------------------------------- #



def zscore(x, window):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

def label_dataset(window_size, variance, min_slope_neg, min_slope_pos, max_lookahead, start_diff_factor, end_perc):
    # Z-score algorithm to find first set of peaks
    labels = zscore(deseasonalized,window_size)<-variance
    print(len(labels == True))

    combined_df = pd.DataFrame(final_ts)
    combined_df.columns = ["value"] 
    combined_df.insert(1, 'label', labels)
    #combined_df.insert(1, 'label', False)
    combined_df.insert(2, 'diff', final_ts.diff())
    combined_df.insert(3, 'deseas_diff', deseasonalized.diff())

    # Mark bonus peaks
    combined_df.loc[combined_df['diff'] < -min_slope_neg, 'label'] = True
    combined_df.loc[combined_df['diff'] > min_slope_pos, 'label'] = True

    labels = combined_df['label']

    # Get individual peak 'groups' from the raw per-sample labels
    labels_shifted = labels.shift(1).fillna(False)
    is_edge = labels != labels_shifted
    edges = pd.DataFrame({'data': labels, 'edge': is_edge})
    peakGroups = edges.loc[edges['edge']==True]

    # Expand each peak group to capture entire peak
    for i, g in peakGroups.groupby(np.arange(len(peakGroups)) // 2):
        peakStart = g.index[0]
        peakEnd = g.index[1]

        realPeakStart = []
        realPeakStartValue = 0

        previousDiff = 0
        n = 0 # limit on how far to look back to find start of peak
        currRow = combined_df.loc[peakStart]

        while (True and n < max_lookahead):
            currDiff = currRow['diff']

            if (abs(currDiff)*start_diff_factor < abs(previousDiff)):
                # Start of peak found
                realPeakStart = currRow
                realPeakStartValue = currRow['value']
                break

            # Check next row
            n += 1
            previousDiff = currDiff
            currRow = combined_df.loc[currRow.name - pd.Timedelta(minutes=RESAMPLE_FREQ)]

        peakMinimum = combined_df.loc[realPeakStart.name:peakEnd, 'value'].min()

        realPeakEnd = []

        n = 0 # limit on how far to look forward to find end of peak
        previousVal = 0
        currRow = combined_df.loc[peakEnd]
        while (True and n < max_lookahead):
            currVal = currRow['value']

            if (currVal-peakMinimum > end_perc * (realPeakStartValue-peakMinimum) or currVal < previousVal):
                # End of peak found
                realPeakEnd = currRow
                break

            # Check next row
            n += 1
            previousVal = currVal
            currRow = combined_df.loc[currRow.name + pd.Timedelta(minutes=RESAMPLE_FREQ)]
        
        # Set all the sample labels between the real peak start and end to True
        combined_df.loc[realPeakStart.name:realPeakEnd.name-pd.Timedelta(minutes=RESAMPLE_FREQ), 'label'] = True

        # Add this peak and its statistics to the peakData dataframe.
        peakData.loc[i] = pd.Series({'group':i, 
                'timespan':(realPeakEnd.name - realPeakStart.name),
                'magnitude': combined_df.loc[realPeakStart.name:realPeakEnd.name, 'value'].max() - combined_df.loc[realPeakStart.name:realPeakEnd.name, 'value'].min(),
                'drop': 100 - (combined_df.loc[realPeakStart.name:realPeakEnd.name, 'value'].min() / combined_df.loc[realPeakStart.name:realPeakEnd.name, 'value'].max() * 100), 
                'shape': combined_df.loc[realPeakStart.name:realPeakEnd.name, 'diff'].mean(),
                'startdate': realPeakStart.name, 
                'enddate': realPeakEnd.name})

    return combined_df['label']



# ---- Perform automatic labelling using predefined parameters ----
labeled_dataset = label_dataset(WINDOW_SIZE, Z_THRESHOLD, BONUS_DIFF_THRES, BONUS_DIFF_THRES, MAXIMUM_LOOKAHEAD, PEAK_START_DIFF_FACTOR, PEAK_END_RECOVER_PERC)
# -----------------------------------------------------------------

# Save peak data to a CSV file
peakData.to_csv(PATH + '_peaks.csv')

# Plot results
plt.plot(deseasonalized)
x = np.where(labeled_dataset == True)[0]
plt.plot(final_ts)
plt.plot(deseasonalized.index[x], deseasonalized.values[x], 'ro')
plt.plot(final_ts.index[x], final_ts.values[x], 'ro')

64801


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# OPTIONAL: Find optimal parameters for the labeling algorithm

In [92]:
import skopt
from sklearn.metrics import confusion_matrix, cohen_kappa_score

def objective(search_params):
    df_real = pd.read_csv(REAL_LABELS_FILE)
    df_pred = label_dataset(search_params[0], search_params[1], search_params[2], search_params[2], MAXIMUM_LOOKAHEAD, search_params[3], search_params[4]).astype(int)
    #tn, fp, fn, tp = confusion_matrix(df_real['label'], df_pred).ravel()
    #f1_score = (tp)/(tp+0.5*(fp+fn))
    #return -1* f1_score
    score = -1* cohen_kappa_score(df_real['label'], df_pred)
    print(score)
    return score

SPACE = [
    skopt.space.Integer(3000, 8000, name='window_size'),
    skopt.space.Real(3, 6.5, name='variability_thres', prior='uniform'),
    skopt.space.Integer(800, 6000, name='bonus_diff_threshold'),
    skopt.space.Real(2, 6, name='start_diff', prior='uniform'),
    skopt.space.Real(0.45, 0.85, name='recovery_perc', prior='uniform')]

def run_optimizer():
    results = skopt.forest_minimize(objective, SPACE, n_calls=100, n_random_starts=10)
    #results = skopt.BayesSearchCV(objective, SPACE, n_calls=300, n_random_starts=10)
    best_auc = -1.0 * results.fun
    best_params = results.x
    print('best result: ', best_auc)
    print('best parameters: ', best_params)

#run_optimizer()

209646
-0.6850802199190285
209646
-0.6587285061487649
209646
-0.6954388594289169
209646
-0.5824562991718014
209646
-0.7275388483883323
209646
-0.6924912215021484
209646
-0.6619068721628225
209646
-0.6606762264038641
209646
-0.35091193935985066
209646
-0.5545102602184173
209646
-0.11031270292688145
209646
-0.7129600721994662
209646
-0.6904753966876928
209646
-0.7062949274595822
209646
-0.6893444789759954
209646
-0.7191818993288465
209646
-0.6981271467857078
209646
-0.7191464576469891
209646
-0.651408132565743
209646
-0.7090799086286721
209646
-0.6884127055979279
209646
-0.7014191278399364
209646
-0.7151942008880381
209646
-0.6836784635837387
209646
-0.7072150362480867
209646
-0.5679091096864637
209646
-0.6566636576711888
209646
-0.6516962596274682
209646
-0.5681287492420696
209646
-0.6659240318251691
209646
-0.34622694801828613
209646
-0.5525873159492372
209646
-0.5739737310733352
209646
-0.6297813909945507
209646
-0.5913850390154196
209646
-0.56582785958823
209646
-0.4783147739943743
2

# Result of the automatic labelling
Shows a confusion matrix and the F-score for the previously performed automatic labelling.
Compared against a manually labeled dataset

In [94]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


df_real = pd.read_csv(REAL_LABELS_FILE)
df_pred = labeled_dataset.astype(int)

tn, fp, fn, tp = confusion_matrix(df_real['label'].astype(int), df_pred).ravel()
f1_score = (tp)/(tp+0.5*(fp+fn))
print(f1_score)
print(cohen_kappa_score(df_real['label'], df_pred))

cm = confusion_matrix(df_real['label'].astype(int), df_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["no peak", "peak"])
disp = disp.plot(include_values=["no peak", "peak"], values_format="")
plt.show()

0.7724631202503353
0.7712466124406792


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …